In [125]:
import pandas as pd

train_data = pd.read_csv(r"C:\Users\njh45\OneDrive\바탕 화면\Kuggle_11기\Project\train_cleaned.csv")

In [126]:
target = 'class' 
features = train_data.drop(columns=[target]).columns

## 타겟 변수의 균형이 1:1에 수렴하므로 증강은 사용X

## LightGBM

In [127]:
# 범주형 변수를 'category'로 변환해야 자동으로 인식함
for col in train_data.select_dtypes(include='object').columns:
    train_data[col] = train_data[col].astype('category')

In [128]:
from sklearn.model_selection import train_test_split

# 8:2 분할
X_train, X_val, y_train, y_val = train_test_split(
    train_data[features], train_data[target], test_size=0.2, random_state=42
    )

In [129]:
import lightgbm as lgb

# 학습 성능 향상을 위해 LightGBM 전용 객체로 변환
train_set = lgb.Dataset(X_train, label=y_train)
val_set = lgb.Dataset(X_val, label=y_val)

In [130]:
from lightgbm import early_stopping, log_evaluation
# 피처 이름 정리
train_columns = X_train.columns.str.replace('[^0-9a-zA-Z_]', '', regex=True)
X_train.columns = train_columns
X_val.columns = train_columns

# 모델 하이퍼파라미터 설정
params = {
    'objective': 'binary',           # 이진 분류 문제의 목표 함수
    'metric': 'binary_logloss',      # 손실 함수
    'learning_rate': 0.05,           # 알파
    'num_leaves': 31,                # 하나의 트리가 가질 수 있는 리프의 최대 개수
    'max_depth': -1,                 # 트리의 최대 깊이 (-1은 제한 없음)
    'min_child_samples': 20,         # 최소 데이터 수, 과적합 방지
    'subsample': 0.8,                # 데이터 샘플링 비율
    'colsample_bytree': 0.8,         # 각 트리별로 선택하는 피처의 비율
    'lambda_l1': 0.0,                # L1 규제 값
    'lambda_l2': 1.0,                # L2 규제 값
    'verbose': -1                    # 출력 메시지 레벨 (-1은 경고와 오류만 표시)
}

# 모델 학습
model = LGBMClassifier(**params)
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(100)
    ]
)

Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.0980781
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.0980781


LGBMClassifier(colsample_bytree=0.8, lambda_l1=0.0, lambda_l2=1.0,
               learning_rate=0.05, metric='binary_logloss', objective='binary',
               subsample=0.8, verbose=-1)

In [131]:
# 최적의 반복 횟수를 확인
print(f"Best Iteration: {model.best_iteration_}")

Best Iteration: 100


In [12]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score
from lightgbm import early_stopping, log_evaluation

# 하이퍼파라미터 튜닝 속 검색 공간 정의
space = {
    'num_leaves': hp.choice('num_leaves', range(20, 150, 10)),
    'max_depth': hp.choice('max_depth', range(3, 15)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'min_child_samples': hp.choice('min_child_samples', range(5, 100, 5)),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'lambda_l1': hp.uniform('lambda_l1', 0.0, 1.0),
    'lambda_l2': hp.uniform('lambda_l2', 0.0, 1.0)
}

# 목적 함수 정의
def objective(params):
    model = LGBMClassifier(
        objective='binary',
        metric='binary_logloss',
        **params
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)], 
        eval_metric='binary_logloss',
        callbacks=[early_stopping(stopping_rounds=50), log_evaluation(100)]
    )
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return {'loss': -accuracy, 'status': STATUS_OK}

# 하이퍼파라미터 최적화 실행
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

print("Best parameters found: ", best)


Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.050535              
Did not meet early stopping. Best iteration is:       
[100]	valid_0's binary_logloss: 0.050535
Training until validation scores don't improve for 50 rounds                     
[100]	valid_0's binary_logloss: 0.0592995                                        
Did not meet early stopping. Best iteration is:                                  
[100]	valid_0's binary_logloss: 0.0592995
Training until validation scores don't improve for 50 rounds                     
[100]	valid_0's binary_logloss: 0.0413397                                        
Did not meet early stopping. Best iteration is:                                  
[99]	valid_0's binary_logloss: 0.0413318
Training until validation scores don't improve for 50 rounds                     
[100]	valid_0's binary_logloss: 0.38682                                          
Did not meet early stopping. Best iteration is:  

In [132]:
# 최적화된 파라미터 적용
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.25461348235478287, 
    'num_leaves': 11,         
    'max_depth': 8,          
    'min_child_samples': 13, 
    'subsample': 0.7712935576056525,     
    'colsample_bytree': 0.5019118752261189, 
    'lambda_l1': 0.22919819506037806,     
    'lambda_l2': 0.11125650624853428,     
}

# 재학습
model = LGBMClassifier(**params)
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(100)
    ]
)

Training until validation scores don't improve for 50 rounds
[100]	valid_0's binary_logloss: 0.0762168
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.0762168


LGBMClassifier(colsample_bytree=0.5019118752261189,
               lambda_l1=0.22919819506037806, lambda_l2=0.11125650624853428,
               learning_rate=0.25461348235478287, max_depth=8,
               metric='binary_logloss', min_child_samples=13, num_leaves=11,
               objective='binary', subsample=0.7712935576056525)

In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

# 검증 데이터에 대한 예측 수행
y_pred = model.predict(X_val)

# 정확도 = 실제 값과 예측이 일치한 비율
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy : {accuracy:.4f}\n")

# F1 스코어 = 정밀도와 재현율의 조화 평균
f1 = f1_score(y_val, y_pred)
print(f"F1 Score : {f1:.4f}\n")

# 정밀도(Precision) = 양성으로 예측한 것들 중 실제로 양성인 비율. 모델의 예측을 신뢰가능
# 재현율(Recall) = 실제 양성 데이터 중에서 모델이 양성으로 예측한 비율. 실제 양성을 잘 놓치지 않음.
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
print(f"Precision : {precision:.4f}\n")
print(f"Recall : {recall:.4f}\n")

# 분산(Confusion Matrix)
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n")
print(conf_matrix)


Accuracy : 0.9846

F1 Score : 0.9859

Precision : 0.9870

Recall : 0.9849

Confusion Matrix:

[[273297   4400]
 [  5132 334233]]


## test_cleaned 적용

In [145]:
test_data = pd.read_csv(r"C:\Users\njh45\OneDrive\바탕 화면\Kuggle_11기\Project\test_cleaned.csv")

In [146]:
# 범주형 변수를 'category'로 변환해야 자동으로 인식함
for col in test_data.select_dtypes(include='object').columns:
    test_data[col] = test_data[col].astype('category')

In [147]:
# 타겟 레이블이 없는 경우 (예측 전용)
X_test = test_data

### 컬럼 타입 일치

In [148]:
for col in train_data.select_dtypes(include='category').columns:
    test_data[col] = test_data[col].cat.set_categories(train_data[col].cat.categories)

In [149]:
# Object 타입의 컬럼을 category로 변경 (이미 변경된 경우 생략 가능)
for col in train_data.select_dtypes(include=['object']).columns:
    train_data[col] = train_data[col].astype('category')
    
for col in test_data.select_dtypes(include=['object']).columns:
    test_data[col] = test_data[col].astype('category')

# 각 범주형 변수의 고유값을 비교
for col in train_data.select_dtypes(include=['category']).columns:
    train_unique_values = set(train_data[col].cat.categories)
    test_unique_values = set(test_data[col].cat.categories)
    
    if train_unique_values != test_unique_values:
        print(f"'{col}' 컬럼에서 train과 test의 고유값이 다릅니다.")
        print(f"Train 고유값: {train_unique_values}")
        print(f"Test 고유값: {test_unique_values}")
    else:
        print(f"'{col}' 컬럼은 train과 test의 고유값이 동일합니다.")

'cap-shape' 컬럼은 train과 test의 고유값이 동일합니다.
'cap-surface' 컬럼은 train과 test의 고유값이 동일합니다.
'cap-color' 컬럼은 train과 test의 고유값이 동일합니다.
'does-bruise-or-bleed' 컬럼은 train과 test의 고유값이 동일합니다.
'gill-attachment' 컬럼은 train과 test의 고유값이 동일합니다.
'gill-spacing' 컬럼은 train과 test의 고유값이 동일합니다.
'gill-color' 컬럼은 train과 test의 고유값이 동일합니다.
'stem-color' 컬럼은 train과 test의 고유값이 동일합니다.
'has-ring' 컬럼은 train과 test의 고유값이 동일합니다.
'ring-type' 컬럼은 train과 test의 고유값이 동일합니다.
'habitat' 컬럼은 train과 test의 고유값이 동일합니다.


In [150]:
# 각 데이터프레임의 데이터 타입 출력
print("Train Data Types:\n", train_data.dtypes)

Train Data Types:
 Unnamed: 0                 int64
class                      int64
cap-diameter             float64
cap-shape               category
cap-surface             category
cap-color               category
does-bruise-or-bleed    category
gill-attachment         category
gill-spacing            category
gill-color              category
stem-height              float64
stem-width               float64
stem-color              category
has-ring                category
ring-type               category
habitat                 category
season                     int64
is_spring_or_fall          int64
dtype: object


In [151]:
print("\nTest Data Types:\n", test_data.dtypes)


Test Data Types:
 Unnamed: 0                 int64
cap-diameter             float64
cap-shape               category
cap-surface             category
cap-color               category
does-bruise-or-bleed    category
gill-attachment         category
gill-spacing            category
gill-color              category
stem-height              float64
stem-width               float64
stem-color              category
has-ring                category
ring-type               category
habitat                 category
season                     int64
is_spring_or_fall          int64
dtype: object


In [152]:
# 학습된 모델을 사용해 예측 수행
y_test_pred = model.predict(X_test)

In [167]:
id = pd.read_csv(r"C:\Users\njh45\OneDrive\바탕 화면\Kuggle_11기\Project\sample_submission.csv")

In [168]:
id = id[['id']]

In [195]:
print(y_test_pred)
print(id)

[0 1 1 ... 1 0 0]
              id
0        3116945
1        3116946
2        3116947
3        3116948
4        3116949
...          ...
2077959  5194904
2077960  5194905
2077961  5194906
2077962  5194907
2077963  5194908

[2077964 rows x 1 columns]


In [196]:
# id의 인덱스를 초기화하여 병합 준비
id = id.reset_index(drop=True)

# y_test_pred를 DataFrame으로 변환하여 동일한 길이로 맞춤
y_test_pred_df = pd.DataFrame(y_test_pred, columns=['class'])

# id와 y_test_pred_df를 인덱스를 무시하고 병합
submission = pd.concat([id, y_test_pred_df], axis=1)

In [199]:
print(submission)

              id  class
0        3116945      0
1        3116946      1
2        3116947      1
3        3116948      1
4        3116949      0
...          ...    ...
2077959  5194904      1
2077960  5194905      1
2077961  5194906      1
2077962  5194907      0
2077963  5194908      0

[2077964 rows x 2 columns]


In [201]:
# class 열의 0과 1을 각각 e와 p로 변환
submission['class'] = submission['class'].map({0: 'e', 1: 'p'})

In [202]:
# CSV 파일로 저장
submission.to_csv(r"C:\Users\njh45\OneDrive\바탕 화면\Kuggle_11기\Project\submission4.csv", index=False)